## PartD: prescriber level opioid dosage sum-> MME >120 indicator  => aggregate to census track level to indicate potential opioid overdose severity

In [1]:
#importing various packages I expect to use
import json
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 
import scipy 
import scipy.stats as stats
import math
from scipy import stats

## Loading all the data into Pandas DataFrames

In [2]:
#data = pandas.read_sas('PartD_Prescriber_PUF_NPI_14-SAS-Infile.sas')
PUFNPI = pd.read_csv('PartD_Prescriber_PUF_NPI_14.txt',sep='\t')
PUFNPI = pd.DataFrame(PUFNPI)
PUFNPIDrug = pd.read_csv('PartD_Prescriber_PUF_NPI_Drug_14.txt',sep='\t')
PUFNPIDrug = pd.DataFrame(PUFNPIDrug)
CincinnatiBuprPhysLoc = pd.read_csv('Physician_Locator_2018-04-30T17-01-58.csv')
CincinnatiBuprPhysLoc = pd.DataFrame(CincinnatiBuprPhysLoc)
StDrug  = pd.read_excel('PartD_Prescriber_PUF_Drug_St_14.xlsx',sheetname='Data',header=2)
StDrug = pd.DataFrame(StDrug)
NtlDrug  = pd.read_excel('PartD_Prescriber_PUF_Drug_Ntl_14.xlsx',sheetname='Data',header=2)
NtlDrug = pd.DataFrame(NtlDrug)
RxCat  = pd.read_excel('PartD_Prescriber_PUF_NPI_14_Drug_Category_Lists.xlsx',sheetname=1,header=2)
RxCat = pd.DataFrame(RxCat)

## The Part D Prescriber PUF is organized by National Provider Identifier (NPI) and drug name and contains information on drug utilization (claim counts and day supply) and total drug costs. The data in the Part D Prescriber PUF cover calendar years 2013 through 2015.
### PUFNPIDrug: The PUF NPI Drug (24M rows x 19 columns, rows indexed at NPI+Drug level) file has drug dispensing information and pattern at the NPI level. It has general metrics such as distinct number of  patients on that drug, drugs prescribes , cost of those prescriptions. Bene count is how many patients are on a specific drug. (This table might need to be matched to another drug table to get drug class information, etc.)
### PUFNPI: While the PUF NPI  table (1M rows x 109 columns) has information regarding the provider. It has aggregate Rx info like total drug cost/days supply/30-day fills, and additionally, address/zip-code, generic vs. brand name Rx meta data. It also includes aggreate data on overal Opioid class. 
  #### This information includes opioid_claim_count, opioid_drug_cost, opioid_day_supply, opioid_bene_count, opioid_prescriber_rate. other info include lis_claim_count (Low Income Subsidized drugs), anti-psych meds info,  race-count of patients for that NPI, and average HCC risk.
  
  ### RxCat has medication names and categories
  
  ### CincinnatiBuprPhysLoc: 

In [3]:
print('PUFNPIDrug :',type(PUFNPIDrug), PUFNPIDrug.shape)
print('PUFNPI :',type(PUFNPI), PUFNPI.shape)
print('RxCat :',type(RxCat), RxCat.shape)
print('StDrug :',type(StDrug), StDrug.shape)
print('NtlDrug :',type(NtlDrug), NtlDrug.shape)
print('CincinnatiBuprPhysLoc :',type(CincinnatiBuprPhysLoc), CincinnatiBuprPhysLoc.shape)


PUFNPIDrug : <class 'pandas.core.frame.DataFrame'> (24121659, 21)
PUFNPI : <class 'pandas.core.frame.DataFrame'> (1072997, 84)
RxCat : <class 'pandas.core.frame.DataFrame'> (116, 2)
StDrug : <class 'pandas.core.frame.DataFrame'> (104071, 18)
NtlDrug : <class 'pandas.core.frame.DataFrame'> (3362, 17)
CincinnatiBuprPhysLoc : <class 'pandas.core.frame.DataFrame'> (175, 11)


In [4]:
print('PUFNPIDrug')
print(PUFNPIDrug.info())
print(PUFNPIDrug.columns)
print('------------------------------------------------------------------')
print('PUFNPI')
print(PUFNPI.info())
print(PUFNPI.columns)
print('------------------------------------------------------------------')
print('RxCat')
print(RxCat.info())
print(RxCat.columns)
print('------------------------------------------------------------------')
print('CincinnatiBuprPhysLoc')
print(CincinnatiBuprPhysLoc.info())
print(CincinnatiBuprPhysLoc.columns)
print('------------------------------------------------------------------')
print('StDrug')
print(StDrug.info())
print(StDrug.columns)
print('------------------------------------------------------------------')
print('NtlDrug')
print(NtlDrug.info())
print(NtlDrug.columns)

PUFNPIDrug
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24121659 entries, 0 to 24121658
Data columns (total 21 columns):
npi                              int64
nppes_provider_last_org_name     object
nppes_provider_first_name        object
nppes_provider_city              object
nppes_provider_state             object
specialty_description            object
description_flag                 object
drug_name                        object
generic_name                     object
bene_count                       float64
total_claim_count                int64
total_30_day_fill_count          float64
total_day_supply                 int64
total_drug_cost                  float64
bene_count_ge65                  float64
bene_count_ge65_suppress_flag    object
total_claim_count_ge65           float64
ge65_suppress_flag               object
total_30_day_fill_count_ge65     float64
total_day_supply_ge65            float64
total_drug_cost_ge65             float64
dtypes: float64(8), int64(3),

In [5]:
print('PUFNPIDrug :', PUFNPIDrug.head())
print('PUFNPI :', PUFNPI.head())
print('RxCat :', RxCat.head())
print('StDrug :', StDrug.head())
print('NtlDrug :', NtlDrug.head())
print('CincinnatiBuprPhysLoc :', CincinnatiBuprPhysLoc.head())

PUFNPIDrug :           npi nppes_provider_last_org_name nppes_provider_first_name  \
0  1952310666                    A'BODJEDI                    ENENGE   
1  1952310666                    A'BODJEDI                    ENENGE   
2  1952310666                    A'BODJEDI                    ENENGE   
3  1952310666                    A'BODJEDI                    ENENGE   
4  1952310666                    A'BODJEDI                    ENENGE   

  nppes_provider_city nppes_provider_state specialty_description  \
0            STAMFORD                   CT            Psychiatry   
1            STAMFORD                   CT            Psychiatry   
2            STAMFORD                   CT            Psychiatry   
3            STAMFORD                   CT            Psychiatry   
4            STAMFORD                   CT            Psychiatry   

  description_flag      drug_name       generic_name  bene_count  \
0                S        ABILIFY       ARIPIPRAZOLE         NaN   
1        

### Limiting the PUFNPIDrug dataset to only data from Cincinnati, OH
### Limiting the PUFNPI dataset to only data from Cincinnatio, OH

In [6]:
CincinnatiPUFNPIDrug = PUFNPIDrug[(PUFNPIDrug['nppes_provider_city']=='CINCINNATI')&(PUFNPIDrug['nppes_provider_state']=='OH')]
CincinnatiPUFNPIDrug.head()
CincinnatiPUFNPI = PUFNPI[(PUFNPI['nppes_provider_city']=='CINCINNATI')&(PUFNPI['nppes_provider_state']=='OH')]
CincinnatiPUFNPI.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,...,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65
16617,1932191681,ABBOTT,CAROL,CINCINNATI,OH,Nurse Practitioner,S,AMOX TR-POTASSIUM CLAVULANATE,AMOXICILLIN/POTASSIUM CLAV,12.0,...,12.0,120,198.75,NaN,*,NaN,*,NaN,NaN,NaN
19880,1568414290,ABBOTT,SUSAN,CINCINNATI,OH,Internal Medicine,S,ALENDRONATE SODIUM,ALENDRONATE SODIUM,11.0,...,39.4,1153,286.70,11.0,NaN,30.0,NaN,39.4,1153.0,286.70
19881,1568414290,ABBOTT,SUSAN,CINCINNATI,OH,Internal Medicine,S,ALLOPURINOL,ALLOPURINOL,NaN,...,24.0,720,100.04,NaN,*,16.0,NaN,24.0,720.0,100.04
19882,1568414290,ABBOTT,SUSAN,CINCINNATI,OH,Internal Medicine,S,AMILORIDE-HYDROCHLOROTHIAZIDE,AMILORIDE/HYDROCHLOROTHIAZIDE,NaN,...,26.0,780,270.94,NaN,*,12.0,NaN,26.0,780.0,270.94
19883,1568414290,ABBOTT,SUSAN,CINCINNATI,OH,Internal Medicine,S,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE,13.0,...,103.0,3090,855.64,13.0,NaN,51.0,NaN,103.0,3090.0,855.64


### Keeping the state and national Rx datasets. The data does not get more granular than state/national. Probably can get rid of them unless we are using as comparison of metrics

## Also adding Treatment Facility  (Address + Facility Type)

In [13]:
BHTreatmentFacility1  = pd.read_excel('Behavioral_Health_Treament_Facility_listing_2018_05_01_021323.xlsx')
BHTreatmentFacility1 = pd.DataFrame(BHTreatmentFacility1)
BHTreatmentFacility2  = pd.read_excel('Behavioral_Health_Treament_Facility_listing_2018_05_01_021337.xlsx')
BHTreatmentFacility2 = pd.DataFrame(BHTreatmentFacility2)
BHTreatmentFacility3  = pd.read_excel('Behavioral_Health_Treament_Facility_listing_2018_05_01_021500.xlsx')
BHTreatmentFacility3 = pd.DataFrame(BHTreatmentFacility3)
frames = [BHTreatmentFacility1,BHTreatmentFacility2,BHTreatmentFacility3]
BHTreatmentFacility = pd.concat(frames)

In [14]:
BHTreatmentFacility.shape

(64401, 18)

In [17]:
print('BHTreatmentFacility')
print(BHTreatmentFacility.info())
print(BHTreatmentFacility.columns)

BHTreatmentFacility
<class 'pandas.core.frame.DataFrame'>
Int64Index: 64401 entries, 0 to 4400
Data columns (total 18 columns):
    name1        64401 non-null object
name2            18972 non-null object
street1          64361 non-null object
street2          21582 non-null object
city             64394 non-null object
state            64401 non-null object
zip              64357 non-null object
zip4             14312 non-null object
county           51975 non-null object
phone            64220 non-null object
intake_prompt    2538 non-null object
intake1          8642 non-null object
intake2          5248 non-null object
website          29751 non-null object
latitude         55104 non-null float64
longitude        55104 non-null float64
type_facility    64401 non-null object
last_update      21972 non-null object
dtypes: float64(2), object(16)
memory usage: 10.6+ MB
None
Index(['    name1', 'name2', 'street1', 'street2', 'city', 'state', 'zip',
       'zip4', 'county', 'phone', 'in

In [28]:
print('BHTreatmentFacility :', BHTreatmentFacility.head())

BHTreatmentFacility :                          name1                  name2               street1  \
0  Walnut Street Health Center                    NaN        1005 Walnut St   
1               Central Clinic  Court Clinic Division   909 Sycamore Street   
2   Dr. Gail M. Hellmann, M.D.                    NaN   909 Sycamore Street   
3                 Court Clinic                    NaN   909 Sycamore Street   
4          Michael  Pegram, NP                    NaN  1000 Sycamore Street   

     street2        city state    zip  zip4    county         phone  \
0        NaN  Cincinnati    OH  45202  1109       NaN  513-357-7320   
1        NaN  Cincinnati    OH  45202   NaN  Hamilton  513-352-1342   
2  Suite 400  Cincinnati    OH  45202   NaN  HAMILTON  513-651-9300   
3        NaN  Cincinnati    OH  45202   NaN  Hamilton  513-352-2111   
4        NaN  Cincinnati    OH  45202   NaN  HAMILTON  513-368-2483   

  intake_prompt       intake1 intake2                       website  \
0    

## Also adding Bupronephrine Treatment Facility  (Address + Services)

In [11]:
BupronephrineTreatmentFacility  = pd.read_excel('Behavioral_Health_Treament_Facility_listing_2018_05_01_021054.xlsx',sheetname='Facilities with service detail')
BupronephrineTreatmentFacilityREF  = pd.read_excel('Behavioral_Health_Treament_Facility_listing_2018_05_01_021054.xlsx',sheetname='service code reference')

In [16]:
BupronephrineTreatmentFacility.shape

(12312, 209)

In [27]:
print('BupronephrineTreatmentFacility')
print(BupronephrineTreatmentFacility.info())
print(BupronephrineTreatmentFacility.columns)


BupronephrineTreatmentFacility
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12312 entries, 0 to 12311
Columns: 209 entries,    name1 to n40 
dtypes: float64(194), int64(1), object(14)
memory usage: 19.6+ MB
None
Index(['   name1', 'name2', 'street1', 'street2', 'city', 'state', 'zip',
       'zip4', 'county', 'phone',
       ...
       'f66', 'f67', 'f70', 'f81', 'f92', 'n13', 'n18', 'n23', 'n24', 'n40 '],
      dtype='object', length=209)


In [20]:
print('BupronephrineTreatmentFacility :', BupronephrineTreatmentFacility.head())

BupronephrineTreatmentFacility :                             name1                               name2  \
0                  Central Clinic               Court Clinic Division   
1                Joseph House Inc                                 NaN   
2    Shelterhouse Volunteer Group                                 NaN   
3  Center for Addiction Treatment                                 NaN   
4                  Family Service  Greater Cinnannti Behav Hlth Servs   

                    street1  street2        city state    zip  zip4    county  \
0       909 Sycamore Street      NaN  Cincinnati    OH  45202   NaN  Hamilton   
1      1526 Republic Street      NaN  Cincinnati    OH  45202   NaN  Hamilton   
2           411 Gest Street  Suite 1  Cincinnati    OH  45203   NaN  Hamilton   
3  830 Ezzard Charles Drive      NaN  Cincinnati    OH  45214   NaN  Hamilton   
4       434 Scott Boluevard      NaN   Covington    KY  41011   NaN    Kenton   

          phone  ...  f66 f67 f70 f81  f9

In [32]:
CincinnatiPUFNPI['description_flag'].isnull()

769        False
783        False
913        False
934        False
1090       False
1131       False
2626       False
2704       False
3109       False
3215       False
3961       False
4820       False
5329       False
5508       False
5562       False
5640       False
6154       False
6169       False
6218       False
6250       False
6453       False
6521       False
6895       False
7411       False
7447       False
7533       False
7615       False
7646       False
7846       False
8597       False
           ...  
1065604    False
1065765    False
1065978    False
1066188    False
1066501    False
1066852    False
1067419    False
1067546    False
1067720    False
1067899    False
1068037    False
1068161    False
1068197    False
1069290    False
1069961    False
1070349    False
1070445    False
1070534    False
1070558    False
1070745    False
1070757    False
1070942    False
1071063    False
1071102    False
1071308    False
1072057    False
1072071    False
1072136    Fal

In [34]:
CincinnatiPUFNPI = CincinnatiPUFNPI.drop(['description_flag','medicare_prvdr_enroll_status','mapd_suppress_flag','mapd_claim_count','mapd_drug_cost','pdp_suppress_flag','pdp_claim_count','pdp_drug_cost','lis_suppress_flag','lis_claim_count','lis_drug_cost','nonlis_suppress_flag','nonlis_claim_count','nonlis_drug_cost','antibiotic_claim_count','antibiotic_drug_cost','antibiotic_bene_count','hrm_ge65_suppress_flag','hrm_claim_count_ge65','hrm_drug_cost_ge65','hrm_bene_ge65_suppress_flag','hrm_bene_count_ge65'],axis=1)

In [35]:
CincinnatiPUFNPI.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
769,1932191681,ABBOTT,CAROL,J,CNP,F,I,8444 WINTON RD,NaN,CINCINNATI,...,14.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.8015
783,1750412615,ABBOTT,DANIEL,E,MD,M,I,222 PIEDMONT AVE.,NaN,CINCINNATI,...,13.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,2.2548
913,1568414290,ABBOTT,SUSAN,B,MD,F,I,4600 MCAULEY PLACE,SUITE 115,CINCINNATI,...,22.0,104.0,NaN,NaN,0.0,0.0,NaN,105.0,21.0,1.1052
934,1346226388,ABBOTTSMITH,CHARLES,W,M.D.,M,I,2123 AUBURN AVE,SUITE 139,CINCINNATI,...,322.0,487.0,35.0,NaN,0.0,0.0,NaN,479.0,56.0,1.3869
1090,1972714251,ABDALLAH,MOUHAMAD,NaN,"MD, PHD",M,I,234 GOODMAN ST,NaN,CINCINNATI,...,49.0,83.0,NaN,NaN,NaN,0.0,NaN,66.0,40.0,2.4137


In [37]:
CincinnatiPUFNPI['Address']=CincinnatiPUFNPI['nppes_provider_street1']+','+CincinnatiPUFNPI['nppes_provider_city']+','+CincinnatiPUFNPI['nppes_provider_state']+','+CincinnatiPUFNPI['nppes_provider_zip5']

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [ ]:
nppes_provider_city                 1072997 non-null object
nppes_provider_zip5                 1072931 non-null float64
nppes_provider_zip4                 950214 non-null float64
nppes_provider_state  